In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import math 
from tensorflow import keras

from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


Uv-Model Testing

In [13]:
UvModel = tf.keras.models.load_model('/gdrive/MyDrive/saved_model/UvRadiation')
# Check its architecture
UvModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, None, 60)          360       
                                                                 
 lstm (LSTM)                 (None, None, 64)          32000     
                                                                 
 lstm_1 (LSTM)               (None, None, 128)         98816     
                                                                 
 dense (Dense)               (None, None, 60)          7740      
                                                                 
 dropout (Dropout)           (None, None, 60)          0         
                                                                 
 dense_1 (Dense)             (None, None, 10)          610       
                                                                 
 dense_2 (Dense)             (None, None, 1)           1

In [3]:
cobacoba3 = np.random.randint(20,150,27) #random numbers for forecasting uv 

In [4]:
#predicting UV
window_size=24
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

def cvrttolist (listy) : 
  return listy.tolist()
def extenddf(df,forecast):
  a= cvrttolist(df)
  b=cvrttolist(forecast)
  a.extend(b)
  return np.array(a) 


def forecastuv (df, hour):
  for nextuv in range(hour) :
    lstm_forecast2 = model_forecast(UvModel, df[..., np.newaxis], window_size)
    lstm_forecast2 = lstm_forecast2[((len(df)-1) - window_size):-1, -1, 0]
    df=extenddf(df,lstm_forecast2)

    lstm_forecast2 = model_forecast(UvModel, df[..., np.newaxis], window_size)
    lstm_forecast2 = lstm_forecast2[((len(df))-1 - window_size):-1, -1, 0]
    df = extenddf(df,lstm_forecast2)

    df=cvrttolist(df)
    a=df[-2]
    df.remove(a)
    df=np.array(df)

  return lstm_forecast2,

forecastuv(cobacoba3,4)


(array([76.35452], dtype=float32),)

### TEMPRATURE Model Testing

In [5]:
Temprature = tf.keras.models.load_model('/gdrive/MyDrive/saved_model/Temprature')
# Check its architecture
Temprature.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, None, 60)          360       
                                                                 
 lstm (LSTM)                 (None, None, 64)          32000     
                                                                 
 lstm_1 (LSTM)               (None, None, 64)          33024     
                                                                 
 dense (Dense)               (None, None, 30)          1950      
                                                                 
 dense_1 (Dense)             (None, None, 10)          310       
                                                                 
 dense_2 (Dense)             (None, None, 1)           11        
                                                                 
 lambda (Lambda)             (None, None, 1)           0

In [7]:
cobacoba2 = np.random.randint(25,35,70) #random trial dataset for temprature
cobacoba2

array([34, 29, 31, 27, 30, 33, 29, 32, 26, 27, 32, 31, 28, 29, 27, 31, 30,
       33, 26, 30, 27, 31, 32, 29, 32, 31, 28, 32, 29, 31, 29, 34, 29, 34,
       32, 33, 26, 31, 26, 34, 28, 30, 25, 25, 25, 31, 29, 34, 31, 32, 31,
       29, 28, 34, 34, 33, 29, 34, 34, 32, 32, 25, 29, 25, 27, 30, 33, 29,
       27, 27])

In [11]:
def cvrttolist (listy) : 
  return listy.tolist()
def extenddf(df,forecast):
  a= cvrttolist(df)
  b=cvrttolist(forecast)
  a.extend(b)
  return np.array(a) 
  
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

    
def forecasttemprature (df, hour):
  for nexttemprature in range(hour) :
    lstm_forecast2 = model_forecast(Temprature, df[..., np.newaxis], window_size)
    lstm_forecast2 = lstm_forecast2[((len(df)-1) - window_size):-1, -1, 0]
    df=extenddf(df,lstm_forecast2)
  
    lstm_forecast2 = model_forecast(Temprature, df[..., np.newaxis], window_size)
    lstm_forecast2 = lstm_forecast2[((len(df)-1) - window_size):-1, -1, 0]
    df=extenddf(df,lstm_forecast2)

    df=cvrttolist(df)
    a=df[-2]
    df.remove(a)
    df=np.array(df)

  return lstm_forecast2


In [12]:
forecasttemprature(cobacoba2,2)

array([25.466871], dtype=float32)